# Joint Production-Inventory

In [1]:
import cvxpy as cp
import numpy as np
import torch
import matplotlib.pyplot as plt

import lropt
from lropt.parameter import Parameter
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

In [2]:
T = 24
I = 3
t = np.arange(1, T+1)
d0 = 1000 * (1 + 0.5*np.sin(np.pi*(t-1)/12))
alpha = np.array([1, 1.5, 2]).reshape((3, 1))
c = alpha * (1 + 0.5*np.sin(np.pi*(t-1)/12))

P = 567
Q = 13600
vmin = 500
vmax = 2000
v0 = 500
theta = 0.2

In [4]:
uncertainty_set = lropt.Polyhedral(c=np.eye(T),d=(1+theta)*d0, lb=(1-theta)*d0)
d = UncertainParameter(T, uncertainty_set=uncertainty_set)
p_0 = cp.Variable((I, T))
p_tau = [cp.Variable((I,T)) for _ in range(T-1)]
p = 0
for t in range(T-1):
    p += p_tau[t]*d[t+1]
p += p_0
inventory_exp = v0 + cp.sum(p) - cp.sum(d)

objective = cp.Minimize(cp.sum(cp.multiply(c, p)))
constraints = [
                p >= 0,
                p <= P,
                cp.sum(p, axis=1) <= Q
                inventory_exp >= vmin,
                inventory_exp <= vmax,
              ]

prob = RobustProblem(objective=objective, constraints=constraints)

ValueError: Cannot broadcast dimensions  (3,) (24,)

In [8]:
print((cp.sum(p, axis=1).shape))
print(p[:,0].shape)
print(Q)

(3,)
(3,)
13600


In [ ]:
prob.solve()